In [106]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from keras import Sequential
from keras.src.layers import Dense, SimpleRNN, LSTM
from keras.src.optimizers import Adam

In [8]:
data = np.sin((0.1 * np.arange(400)) ** 2)

In [9]:
px.line(data)

In [23]:
T = 10
D = 1
X = []
Y = []

for i in range(len(data) - T):
    x = data[i:i + T]
    y = data[i + T]

    X.append(x)
    Y.append(y)

X = np.array(X)
Y = np.array(Y)
N = len(X)
print(f"X.shape: {X.shape}\nY.shape: {Y.shape}")

X.shape: (390, 10)
Y.shape: (390,)


In [29]:
# Autoregressive linear model
model = Sequential()
model.add(Dense(1, input_shape=(T,)))

model.compile(optimizer=Adam(learning_rate=0.01), loss="mse")

r = model.fit(X[:-N // 2], Y[:-N // 2], epochs=100, validation_data=(X[-N // 2:], Y[-N // 2:]), batch_size=1)

Epoch 1/100
195/195 [==============================] - 2s 7ms/step - loss: 1.2047 - val_loss: 0.5647
Epoch 2/100
195/195 [==============================] - 1s 4ms/step - loss: 0.5680 - val_loss: 0.5539
Epoch 3/100
195/195 [==============================] - 1s 5ms/step - loss: 0.5190 - val_loss: 0.5953
Epoch 4/100
195/195 [==============================] - 1s 5ms/step - loss: 0.5161 - val_loss: 0.6054
Epoch 5/100
195/195 [==============================] - 1s 4ms/step - loss: 0.5221 - val_loss: 0.6010
Epoch 6/100
195/195 [==============================] - 1s 5ms/step - loss: 0.5205 - val_loss: 0.6028
Epoch 7/100
195/195 [==============================] - 1s 4ms/step - loss: 0.5255 - val_loss: 0.5904
Epoch 8/100
195/195 [==============================] - 1s 4ms/step - loss: 0.5191 - val_loss: 0.5990
Epoch 9/100
195/195 [==============================] - 1s 4ms/step - loss: 0.5286 - val_loss: 0.5936
Epoch 10/100
195/195 [==============================] - 1s 5ms/step - loss: 0.5241 - val_lo

In [31]:
def draw_loss(r):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=r.epoch, y=r.history["loss"], name="loss"))
    fig.add_trace(go.Scatter(x=r.epoch, y=r.history["val_loss"], name="val_loss"))
    fig.show()


draw_loss(r)

In [54]:
outputs = model.predict(X)

fig = px.line(Y, title="Linear Regression")
fig.add_scatter(y=outputs.ravel(), mode='lines')
fig.show()

13/13 [==============================] - 0s 2ms/step


In [70]:
predictions = []

last_X = X[-N // 2]

while len(predictions) < 200:
    p = model.predict(last_X.reshape(-1, T), verbose=0)[0, 0]
    predictions.append(p)

    last_X = np.roll(last_X, -1)
    last_X[-1] = p

predictions = np.array(predictions)

fig = px.line(Y[-N // 2:], title="Multi-step forecast")
fig.add_scatter(y=predictions, mode="lines")
fig.show()

In [100]:
# SimpleRNN model
X = X.reshape(-1, T, D)

model = Sequential()
model.add(SimpleRNN(10, input_shape=(T, D)))
model.add(Dense(1))

model.compile(optimizer=Adam(learning_rate=0.05), loss="mse")

r = model.fit(X[:-N // 2], Y[:-N // 2], epochs=125, validation_data=(X[-N // 2:], Y[-N // 2:]), batch_size=32)

Epoch 1/125
7/7 [==============================] - 2s 58ms/step - loss: 0.6203 - val_loss: 0.6272
Epoch 2/125
7/7 [==============================] - 0s 15ms/step - loss: 0.5459 - val_loss: 0.5561
Epoch 3/125
7/7 [==============================] - 0s 14ms/step - loss: 0.5106 - val_loss: 0.5543
Epoch 4/125
7/7 [==============================] - 0s 14ms/step - loss: 0.4623 - val_loss: 0.5608
Epoch 5/125
7/7 [==============================] - 0s 15ms/step - loss: 0.4307 - val_loss: 0.5041
Epoch 6/125
7/7 [==============================] - 0s 15ms/step - loss: 0.3878 - val_loss: 0.5588
Epoch 7/125
7/7 [==============================] - 0s 15ms/step - loss: 0.3963 - val_loss: 0.5350
Epoch 8/125
7/7 [==============================] - 0s 14ms/step - loss: 0.3509 - val_loss: 0.4987
Epoch 9/125
7/7 [==============================] - 0s 14ms/step - loss: 0.3523 - val_loss: 0.4838
Epoch 10/125
7/7 [==============================] - 0s 15ms/step - loss: 0.3183 - val_loss: 0.4290
Epoch 11/125
7/7 [=

In [101]:
draw_loss(r)

In [105]:
outputs = model.predict(X)

plot = px.line(Y, title="One-step forecast SimpleRNN")
plot.add_scatter(y=outputs.ravel(), mode='lines')
plot.show()

13/13 [==============================] - 0s 6ms/step


In [107]:
preds = []
last_x = X[-N // 2]

while len(preds) < 250:
    p = model.predict(last_x.reshape(-1, T, D), verbose=0)
    preds.append(p)

    last_x = np.roll(last_x, -1)
    last_x[-1] = p

fig = px.line(Y[-N // 2:], title="Multi-step forecast SimpleRNN")
fig.add_scatter(y=np.array(preds).ravel(), mode="lines")
fig.show()

In [125]:
# LSTM model
X = X.reshape(-1, T, D)

model = Sequential()
model.add(LSTM(10, input_shape=(T, D)))
model.add(Dense(1))

model.compile(optimizer=Adam(learning_rate=0.05), loss="mse")

r = model.fit(X[:-N // 2], Y[:-N // 2], epochs=200, validation_data=(X[-N // 2:], Y[-N // 2:]), batch_size=16)

Epoch 1/200
13/13 [==============================] - 3s 61ms/step - loss: 0.5588 - val_loss: 0.4995
Epoch 2/200
13/13 [==============================] - 0s 13ms/step - loss: 0.5337 - val_loss: 0.5269
Epoch 3/200
13/13 [==============================] - 0s 13ms/step - loss: 0.5117 - val_loss: 0.5281
Epoch 4/200
13/13 [==============================] - 0s 15ms/step - loss: 0.4895 - val_loss: 0.5833
Epoch 5/200
13/13 [==============================] - 0s 13ms/step - loss: 0.3937 - val_loss: 0.4241
Epoch 6/200
13/13 [==============================] - 0s 15ms/step - loss: 0.2295 - val_loss: 0.3500
Epoch 7/200
13/13 [==============================] - 0s 17ms/step - loss: 0.1600 - val_loss: 0.1281
Epoch 8/200
13/13 [==============================] - 0s 15ms/step - loss: 0.1087 - val_loss: 0.1002
Epoch 9/200
13/13 [==============================] - 0s 14ms/step - loss: 0.0755 - val_loss: 0.1676
Epoch 10/200
13/13 [==============================] - 0s 13ms/step - loss: 0.0644 - val_loss: 0.1088

In [126]:
draw_loss(r)

In [127]:
p = model.predict(X)

fig = px.line(Y, title="One-step forecast LSTM")
fig.add_scatter(y=p.ravel())
fig.show()

13/13 [==============================] - 1s 4ms/step


In [128]:
preds = []
x_to_prediction = X[-N // 2]

while len(preds) < 300:
    p = model.predict(x_to_prediction.reshape(-1, T, D), verbose=0)
    preds.append(p)

    x_to_prediction = np.roll(x_to_prediction, -1)
    x_to_prediction[-1] = p

fig = px.line(Y[-N // 2:], title="Multi-step forecast LSTM")
fig.add_scatter(y=np.array(preds).ravel())
fig.show()